In [6]:
import pandas as pd
import numpy as np
import os 

from sklearn.model_selection import train_test_split


In [7]:
PROJECT_DIR = r'C:\Users\user\Desktop\flight-predictor'
DATA_DIR = 'data'


## 1. function to read csv files:


In [8]:
def read_dataset(file_name):
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    try:
        df = pd.read_csv(file_path)
        return df
    except FileNotFoundError:
        print(f"File '{file_name}' not found in directory.")
        return None

In [9]:
flights = read_dataset('flight_price.csv')

In [10]:
flights.head(10)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
5,SpiceJet,24/06/2019,Kolkata,Banglore,CCU → BLR,09:00,11:25,2h 25m,non-stop,No info,3873
6,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,18:55,10:25 13 Mar,15h 30m,1 stop,In-flight meal not included,11087
7,Jet Airways,01/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:00,05:05 02 Mar,21h 5m,1 stop,No info,22270
8,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:55,10:25 13 Mar,25h 30m,1 stop,In-flight meal not included,11087
9,Multiple carriers,27/05/2019,Delhi,Cochin,DEL → BOM → COK,11:25,19:15,7h 50m,1 stop,No info,8625


In [11]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


#### Missing Values
-Column `Route` and `Total_Stops` are missing a value each


# 2. Premilinary Data Analysis: 

### Data type

In [12]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

### Analysis on Data Type
- the data type of `Date_Of_Journey` shoud be datetime.
- the data type of `Dep_Time` shoud be datetime.
- the data type of `Arrival_Time` shoud be datetime.
- the data type of `Duration` is mixed. Should only be kept in minutes.
- delete route column as it is of no use.

### Duplicated Values

In [13]:
flights.duplicated().sum()


220

#### 220 VALUES IN THE DATA ARE DUPLICATED


In [14]:
(
    flights.
    loc[flights.duplicated(keep=False)]
    .sort_values(['Airline','Date_of_Journey','Source','Destination'])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


# 3. Detailed Analysis


### 3.1 Airline

In [15]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

- Vistara Premium economies, Multiple Carriers Premium economy and Jet Airways Business are not correct airline entries

### 3.2 Date_of_Journey

In [16]:
pd.to_datetime(flights.Date_of_Journey, dayfirst = True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

In [17]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

### 3.3 Source

In [18]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### 3.4 Destination

In [19]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### 3.5 Dep_time

In [20]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [21]:
(

    flights.
    Dep_Time.
    loc[lambda ser: ser.str.contains("[^0-9:]") ]
)

Series([], Name: Dep_Time, dtype: object)

In [22]:
pd.to_datetime(flights.Dep_Time).dt.time

C:\Users\user\AppData\Local\Temp\ipykernel_17072\1056054517.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### 3.6 Arrival_Time

In [23]:
flights.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [24]:
(
    flights
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(" ", n=1)
    .str.get(1)
    .unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

### 3.7 Duration

In [25]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("m")]
    .unique()
    
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [26]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("h")]
    # .unique()
    
)


6474    5m
Name: Duration, dtype: object

###
- 5 min flight that is clearly a wrong entry so need to be removed.
- row 6474 needs to be deleted.

In [27]:
flights.iloc[6474]

Airline                        Air India
Date_of_Journey                6/03/2019
Source                            Mumbai
Destination                    Hyderabad
Route              BOM → GOI → PNQ → HYD
Dep_Time                           16:50
Arrival_Time                       16:55
Duration                              5m
Total_Stops                      2 stops
Additional_Info                  No info
Price                              17327
Name: 6474, dtype: object

In [40]:
(
    flights.
    Duration
    .drop(index=6474)
    .str.split(" ", expand=True)
    .set_axis(["hour", "minute"], axis=1)
    .assign(
        hour =lambda df_:(
            df_
            .hour
            .str.replace("h", "" )
            .astype(int)
            .mul(60)
        ),
            minute=lambda df_:(
                df_
                .minute
                .str.replace("m", "")
                .fillna("0")
                .astype(int)
                
                
            )
    )
    .sum(axis=1)
    .rename("duration_minutes")
    .to_frame()
    .join(flights.Duration.drop(index=6474))
)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


### 3.8 total_stops

In [29]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [41]:
(
    flights
    .Total_Stops
    .replace("non-stop", "0")
    .str.replace(" stops?", "", regex=True)
    .pipe(lambda ser: pd.to_numeric(ser))
    
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

### Additional_info

In [ ]:
flights.Additional_Info.unique()

In [ ]:
(
    flights
    .Additional_Info
    .str.replace("No Info","No info")
    .unique()
)

# 4. Data Cleaning

In [44]:
def convert_to_minutes(ser):
    return(
        ser
        .str.split(" ", expand=True)
    .set_axis(["hour", "minute"], axis=1)
    .assign(
        hour =lambda df_:(
            df_
            .hour
            .str.replace("h", "" )
            .astype(int)
            .mul(60)
        ),
            minute=lambda df_:(
                df_
                .minute
                .str.replace("m", "")
                .fillna("0")
                .astype(int) 
            )
    )
    .sum(axis=1)
    )
    
    

In [ ]:
def data_cleaning(df):
    return (
        df
        .drop(index=[6474])
        .drop_duplicates()
         .assign(**{
             col: df[col].str.lower()
             for col in df.select_dtypes(include = 'O').columns
         }         
        )
        .rename(columns= str.lower)
        .assign(
           airline = lambda df_: (
               df_
               .airline
               .str.replace(" Premium economy", "")
               .str.replace(" Business", "")
               .str.title()
           ),
            date_of_journey = lambda df_: pd.to_datetime(flights.Date_of_Journey, dayfirst = True),
            dep_time=lambda df_: pd.to_datetime(df.Dep_Time).dt.time,
            arrival_time = lambda df_: pd.to_datetime(df.Arrival_Time).dt.time,
            duration = lambda df_: df_.duration.pipe(convert_to_minutes),
            total_stops = lambda df_:
            (
                df_
                .total_stops
                .replace("non-stop", "0")
                .str.replace(" stops?", "", regex=True)
                .pipe(lambda ser: pd.to_numeric(ser))
            ),
            additional_info=lambda df_:(
                df_.additional_info
                .str.replace("No Info","No info")
            ) 
        )
        .drop(columns="route")
        
    )


In [45]:
data_cleaning(flights)

C:\Users\user\AppData\Local\Temp\ipykernel_17072\131703206.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time=lambda df_: pd.to_datetime(df.Dep_Time).dt.time,
C:\Users\user\AppData\Local\Temp\ipykernel_17072\131703206.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time = lambda df_: pd.to_datetime(df.Arrival_Time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,banglore,new delhi,22:20:00,01:10:00,170,0.0,no info,3897
1,Air India,2019-05-01,kolkata,banglore,05:50:00,13:15:00,445,2.0,no info,7662
2,Jet Airways,2019-06-09,delhi,cochin,09:25:00,04:25:00,1140,2.0,no info,13882
3,Indigo,2019-05-12,kolkata,banglore,18:05:00,23:30:00,325,1.0,no info,6218
4,Indigo,2019-03-01,banglore,new delhi,16:50:00,21:35:00,285,1.0,no info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,kolkata,banglore,19:55:00,22:25:00,150,0.0,no info,4107
10679,Air India,2019-04-27,kolkata,banglore,20:45:00,23:20:00,155,0.0,no info,4145
10680,Jet Airways,2019-04-27,banglore,delhi,08:20:00,11:20:00,180,0.0,no info,7229
10681,Vistara,2019-03-01,banglore,new delhi,11:30:00,14:10:00,160,0.0,no info,12648


In [48]:
clean_data = data_cleaning(flights)

C:\Users\user\AppData\Local\Temp\ipykernel_17072\131703206.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time=lambda df_: pd.to_datetime(df.Dep_Time).dt.time,
C:\Users\user\AppData\Local\Temp\ipykernel_17072\131703206.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time = lambda df_: pd.to_datetime(df.Arrival_Time).dt.time,


# 5. Spilt Data Into Subsets 

In [57]:
clean_data.shape

(10462, 10)

#### not taking entire 10462 rows since more computation will be required on sagemaker when training tha data.

In [65]:
flights_final = clean_data.sample(5000)

In [66]:
X = flights_final.drop(columns="price")
y = flights_final.price.copy()

In [67]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(3200, 9) (3200,)
(800, 9) (800,)
(1000, 9) (1000,)


# 6. Exporting Subsets

In [68]:
def export_data(X, y, name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)

    X.join(y).to_csv(file_path, index=False)

    return pd.read_csv(file_path).head()

In [69]:
export_data(X_train, y_train, "train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air Asia,2019-04-18,kolkata,banglore,19:55:00,22:25:00,150,0.0,no info,4409
1,Indigo,2019-06-09,delhi,cochin,17:05:00,22:15:00,310,1.0,no info,6493
2,Jet Airways,2019-03-27,delhi,cochin,11:30:00,04:25:00,1015,1.0,in-flight meal not included,6017
3,Spicejet,2019-06-09,kolkata,banglore,06:55:00,09:30:00,155,0.0,no check-in baggage included,3841
4,Jet Airways,2019-03-03,delhi,cochin,05:40:00,04:25:00,1365,2.0,no info,19266


In [70]:
export_data(X_val, y_val, "val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-27,delhi,cochin,19:10:00,19:45:00,1475,2.0,no info,10929
1,Jet Airways,2019-03-12,banglore,new delhi,22:55:00,16:10:00,1035,1.0,no info,13817
2,Jet Airways,2019-04-09,delhi,cochin,23:05:00,12:35:00,810,2.0,no info,9483
3,Air Asia,2019-05-18,kolkata,banglore,19:55:00,22:25:00,150,0.0,no info,5989
4,Multiple Carriers,2019-05-27,delhi,cochin,07:00:00,19:00:00,720,1.0,no info,10407


In [72]:
export_data(X_test, y_test, "test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Multiple Carriers,2019-03-21,delhi,cochin,10:45:00,18:50:00,485,1.0,no info,11622
1,Jet Airways,2019-05-01,kolkata,banglore,20:25:00,14:25:00,1080,1.0,no info,11467
2,Air Asia,2019-04-03,banglore,delhi,23:55:00,02:45:00,170,0.0,no info,4483
3,Indigo,2019-06-09,delhi,cochin,05:10:00,10:05:00,295,1.0,no info,6496
4,Jet Airways,2019-05-12,kolkata,banglore,06:30:00,12:00:00,330,1.0,in-flight meal not included,8586
